In [ ]:
# make sure to install these packages before running:
# pip install pandas
# pip install sodapy

import requests
import pandas as pd
import numpy as np
from sodapy import Socrata


## Create a Socrata account

You'll need to create a Socrata account at https://data.seattle.gov/signup to be given an API token.

In [ ]:
token = 'your API token'

In [ ]:
#set the client to data.seattle.gov
client = Socrata("data.seattle.gov", token)

## Get the data from each individual counter
Each bike counter has an API endpoint used for access to the data.

In [ ]:
# First 100,000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
#I couldn't get the code to work without setting some sort of limit
fremont = client.get("65db-xm6k", limit = 100000)

# Convert to pandas DataFrame
fremont = pd.DataFrame.from_records(fremont)

#now do the rest of the bike counters
fiftyeight = client.get("47yq-6ugv", limit = 100000)
fiftyeight = pd.DataFrame.from_records(fiftyeight)

bgt = client.get("2z5v-ecg8", limit = 100000)
bgt = pd.DataFrame.from_records(bgt)

thirtynine = client.get("3h7e-f49s", limit = 100000)
thirtynine = pd.DataFrame.from_records(thirtynine)

broadway = client.get("j4vh-b42a", limit = 100000)
broadway = pd.DataFrame.from_records(broadway)

twentysix = client.get("mefu-7eau", limit = 100000)
twentysix = pd.DataFrame.from_records(twentysix)

elliotbay = client.get("4qej-qvrz", limit = 100000)
elliotbay = pd.DataFrame.from_records(elliotbay)

bgt = client.get("2z5v-ecg8", limit = 100000)
bgt = pd.DataFrame.from_records(bgt)

mtsw = client.get("u38e-ybnc", limit = 100000)
mtsw = pd.DataFrame.from_records(mtsw)

cheifsealth = client.get("uh8h-bme7", limit = 100000)
cheifsealth = pd.DataFrame.from_records(cheifsealth)

spokane = client.get("upms-nr8w", limit = 100000)
spokane = pd.DataFrame.from_records(spokane)

second = client.get("avwm-i8ym", limit = 100000)
second = pd.DataFrame.from_records(second)

## Format each dataset

Note as of January 2020: A lot of bike counters have data inconsistencies. Cheaf Sealth is notoriously unreliable, and the greenways (thirtysix, fiftyeight, and twentysix) are also not as reliable, especially twentysix. Fremont and Spokane St are going to be the most reliable, with the Burke Gilman at 70th and Elliot Bay fairly reliable. MTS Greenway West is usually ok, although there are some months with data missing, it should be fairly obvious when it's not working.

In [ ]:
#get the columns in the right order so we can merge later
fiftyeight = fiftyeight[['date','east','west','nw_58th_st_greenway_st_22nd_ave_nw_total']]
#add a location field so we remember where these counts came from
fiftyeight['loc'] = "nw_58th_st_greenway_st_22nd_ave_nw"

In [ ]:
bgt = bgt[['date','bike_north','bike_south','bgt_north_of_ne_70th_total']]
bgt['loc'] = "bgt_north_of_ne_70th"

In [ ]:
fremont = fremont[['date','fremont_bridge_nb','fremont_bridge_sb']]
#this file does not already have a totals column so we add one
fremont['bike_tot'] = fremont['fremont_bridge_nb'].astype(float) + fremont['fremont_bridge_sb'].astype(float)
fremont['loc'] = "fremont_bridge"


In [ ]:
thirtynine = thirtynine[['date','north','south','_39th_ave_ne_greenway_at_ne_62nd_st_total']]
thirtynine['loc'] = "39th_ave_ne_greenway_at_ne_62nd_st"

In [ ]:
broadway = broadway[['date','nb','sb','broadway_cycle_track_north_of_e_union_st_total']]
broadway['loc'] = "broadway_cycle_track_north_of_e_union_s"

In [ ]:
twentysix = twentysix[['date','north','south','_26th_ave_sw_greenway_at_sw_oregon_st_total']]
twentysix['loc'] = "26th_ave_sw_greenway_at_sw_oregon_st"

In [ ]:
elliotbay = elliotbay[['date','bike_north','bike_south']]
elliotbay['biketot'] = elliotbay['bike_north'].astype(float) + elliotbay['bike_south'].astype(float)
elliotbay['loc'] = "elliotbay_myrtle_edwards"

In [ ]:
mtsw = mtsw[['date','bike_east','bike_west','mts_trl_west_of_i_90_bridge_total']]
mtsw['loc'] = "mountains_to_sound_west_of_i90"

In [ ]:
cheifsealth = cheifsealth[['date','bike_north','bike_south']]
cheifsealth['biketot'] = cheifsealth['bike_north'].astype(float) + cheifsealth['bike_south'].astype(float)
cheifsealth['loc'] = "cheifsealth"

In [ ]:
spokane = spokane[['date','east','west','spokane_st_bridge_total']]
spokane['loc'] = "spokane_st_bridge"

In [ ]:
second = second[['date','nb','sb','_2nd_ave_cycletrack']]
second['loc'] = "2nd_ave_cycletrack"

## Merge the datasets

bikedir1 is either north or east; bikedir2 is either south or west. 
You could also change the format to have 4 columns: date, bikecount, direction, location which may make it easier for some types of analysis. If you are not very familiar with Seattle's bike counters, you may want to reformat so you don't have to remember if the counter is oriented east-west or north-south.

In [ ]:
allbikes = pd.DataFrame(np.concatenate( (fiftyeight.values,fremont.values,bgt.values,thirtynine.values,broadway.values,twentysix.values,elliotbay.values,mtsw.values,cheifsealth.values,spokane.values,second.values)))
allbikes.columns = ['date','bikedir1','bikedir2','biketot','loc']
allbikes

## Save to file

In [ ]:
allbikes.to_csv("X:\\YourFile\\allbikes.csv")